In [ ]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
import mysql.connector as mysql
import concurrent.futures
from mysql.connector import errorcode
import threading

import pandas as pd
from mysql.connector import pooling
from mysql.connector import Error

max_threads = 30
semaphore = threading.Semaphore(max_threads)




headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}


urls1 = ['https://ickala.com/43-ic-parts', 'https://ickala.com/86-microwave-parts', 'https://ickala.com/95-optocoupler', 'https://ickala.com/1076-transistor-diode-thyristor', 'https://ickala.com/593-passive-components', 'https://ickala.com/1038-modules', 'https://ickala.com/79-crystal-parts', 'https://ickala.com/1078-display-led', 'https://ickala.com/1074-electromechanical', 'https://ickala.com/606-all-conectors', 'https://ickala.com/1073-circuit-protection', 'https://ickala.com/1077-power-product', 'https://ickala.com/115-instruments-tools', 'https://ickala.com/592-tools-supplies', 'https://ickala.com/1079-electromechanical', 'https://ickala.com/928-miners-components', 'https://ickala.com/1075-wire-cable', 'https://ickala.com/787-industrial-electricity', 'https://ickala.com/972-electronic-devices', 'https://ickala.com/1237-oring-packing', 'https://ickala.com/1178-electrical-pack','https://ickala.com/109-sensor-parts']


Main_links = ['https://ickala.com/319-programmers?id_category=319&n=20']


timeout = 5
for main_url in urls1:
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(executable_path="M:\chromedriver_win32\chromedriver.exe")
    driver.maximize_window()
    driver.get(main_url)
    driver.find_element('xpath','//*[@id="pagination_bottom"]/form/div/button/span').click()
    my_currentUrl = driver.current_url
    print(my_currentUrl)
    Main_links.append(my_currentUrl.strip())
    driver.close()





def fetch_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    product_links = [link.get("href") for link in soup.find_all("a", class_="product-name")]
    return product_links


with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = [executor.submit(fetch_url, url) for url in Main_links]


# Wait for all the results and extract the product links
product_links = []
for f in concurrent.futures.as_completed(results):
    product_links += f.result()

print(len(product_links))


N_of_Links_extracted = 0
numberOfnoQuantity = 0

Titles = []
Prices = []
Quanitities = []
Categories = []
Links = []
Packages = []


def scrape_each_product(product_link):
    global N_of_Links_extracted
    global  numberOfnoQuantity

    try:
        page1 = requests.get(product_link, headers=headers)

        soup3 = BeautifulSoup(page1.content, "html.parser")

        title = soup3.find('h5').get_text()

        price = soup3.find(id='our_price_display').get_text()
    
        quantity = soup3.find(id='quantityAvailable').get_text()


        try:

            package = soup3.findAll('span', {'class': 'feature_value'})
            Package_Existence = soup3.findAll('span', {'class': 'feature_name'})
            if 'Package :'in str(Package_Existence):
                package_TMP = ','.join([package[2].get_text(), package[1].get_text()])
                Packages.append(package_TMP.strip())
            else:
                Packages.append('None')
        except:
            Packages.append('None')

        link_parts = product_link.split('/')

        Subcategory = link_parts[3]
        Category_TMP = Subcategory

        Titles_TMP = title.strip()

        Prices_TMP = (price.strip().replace('ریال', '').strip())
        Prices_TMP = (Prices_TMP.replace(',',''))
        Prices_TMP = int(Prices_TMP)
        Quanitities_TMP = (quantity.strip())

        Link_TMP = product_link.strip()

        Titles.append(Titles_TMP)
        Prices.append(Prices_TMP)
        Quanitities.append(Quanitities_TMP)
        Categories.append(Category_TMP)
        Links.append(Link_TMP)


        try:
            connection_pool = pooling.MySQLConnectionPool(pool_name="pynative_pool",
            pool_size=3,
            pool_reset_session=True,
            host='localhost',
            database='shop',
            user='root',
            password='00980351zmz')

            # Get connection object from a pool
            with connection_pool.get_connection() as cnx:
                 with cnx.cursor() as cursor:
                    add_Product = ("INSERT INTO products "
                                   "(Title, Packages, Price, Category, Quaintities, Link) "
                                   "VALUES (%s, %s, %s, %s, %s, %s)")
                    Product_data = (Titles_TMP, package_TMP, Prices_TMP, Category_TMP, Quaintities_TMP, Link_TMP)
                    cursor.execute(add_Product, Product_data)
                    cnx.commit()
                    print(f'Links Extracted: {N_of_Links_extracted}')

        except Error as e:
            print("Error while connecting to MySQL using Connection pool ", e)
        finally:
            # closing database connection.
            if cnx.is_connected():
                cursor.close()
                connection_pool.put_connection(cnx)
                print("MySQL connection is closed")


        #
        # main_dict = {'Links': Links, 'Category': Categories, 'Product_name': Titles, 'Price': Prices,'Package': Packages , 'Quantity': Quanitities}
        # df = pd.DataFrame(main_dict)
        # df.to_csv('Ickala-data.csv')
        #
        # N_of_Links_extracted += 1
        # print(f'Links Extracted: {N_of_Links_extracted}')


        # add_Product = ("INSERT INTO products "
        #                 "(Title, Packages, Price, Category, Quaintities, Link) "
        #                 "VALUES (%s, %s, %s, %s, %s)")
        # Product_data = (Titles_TMP, Prices_TMP, Category_TMP, Quanitities_TMP, Link_TMP)
        #
        #
        # try:
        #     cursor.execute(add_Product, Product_data)
        #     cnx.commit()
        #     print("Bilakh")
        #
        # except:
        #     print("Duplicate")



    except:
        numberOfnoQuantity += 1
        print(f'Links Not Extracted: {numberOfnoQuantity}')



with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor1:
    # Submit tasks to the thread pool
    futures1 = [executor1.submit(scrape_each_product, product_link) for product_link in product_links]

    # Wait for all tasks to complete
    for future1 in concurrent.futures.as_completed(futures1):
        # Get any exceptions raised by the task
        try:
            future1.result()
        except Exception as e:
            print(f"Task raised an exception: {e}")



#
# cursor.close()
# cnx.close()

C:\Users\Hamidreza\AppData\Local\Temp\ipykernel_15552\2219483735.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="M:\chromedriver_win32\chromedriver.exe")


https://ickala.com/43-ic-parts?id_category=43&n=5819
https://ickala.com/86-microwave-parts?id_category=86&n=502
https://ickala.com/95-optocoupler?id_category=95&n=291
https://ickala.com/1076-transistor-diode-thyristor?id_category=1076&n=3302
https://ickala.com/593-passive-components?id_category=593&n=4756
https://ickala.com/1038-modules?id_category=1038&n=335
https://ickala.com/79-crystal-parts?id_category=79&n=317
https://ickala.com/1078-display-led?id_category=1078&n=207
https://ickala.com/1074-electromechanical?id_category=1074&n=572
https://ickala.com/606-all-conectors?id_category=606&n=967
https://ickala.com/1073-circuit-protection?id_category=1073&n=521
https://ickala.com/1077-power-product?id_category=1077&n=1375
https://ickala.com/115-instruments-tools?id_category=115&n=206
https://ickala.com/592-tools-supplies?id_category=592&n=485
https://ickala.com/1079-electromechanical?id_category=1079&n=81
https://ickala.com/928-miners-components?id_category=928&n=126
https://ickala.com/1

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import mysql.connector as mysql
import concurrent.futures
from mysql.connector import errorcode
import threading

cnx = mysql.connect(host='localhost',user='root',
                    password='00980351zmz',
                    database='shop')

cursor = cnx.cursor()



headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
# def change_tcp_timed_wait_delay():
#     # Open the Windows registry key for editing
#     key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, r"SYSTEM\CurrentControlSet\Services\TCPIP\Parameters", 0, winreg.KEY_ALL_ACCESS)
#
#     # Set the value of TcpTimedWaitDelay to 30 (decimal)
#     winreg.SetValueEx(key, "TcpTimedWaitDelay", 0, winreg.REG_DWORD, 30)
#
#     # Close the registry key
#     winreg.CloseKey(key)
#
# # Call the function to change TcpTimedWaitDelay
# change_tcp_timed_wait_delay()

urls = ["https://ickala.com/43-ic-parts?id_category=43&n=5468"
        ]
N_of_Links_extracted = 0
numberOfnoQuantity = 0


def fetch_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    product_links = [link.get("href") for link in soup.find_all("a", class_="product-name")]
    return product_links

# List of URLs to scrape


# Use a ThreadPoolExecutor to make the requests in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
     results = [executor.submit(fetch_url, url) for url in urls]

# Wait for all the results and extract the product links

product_links = []
for f in concurrent.futures.as_completed(results):
    product_links += f.result()

print(len(product_links))
# Print the product links

for p in product_links:
    try:
        page1 = requests.get(p, headers=headers)

        soup3 = BeautifulSoup(page1.content, "html.parser")

        soup4 = BeautifulSoup(soup3.prettify(), "html.parser")

        title = soup4.find('h5').get_text()

        price = soup4.find(id='our_price_display').get_text()

        quantity = soup4.find(id='quantityAvailable').get_text()
        try:
            package = soup3.findAll('span', {'class': 'feature_value'})
            package_TMP = package[2].get_text()
            add_package_TMP = ("INSERT INTO products "
                               "(Package)"
                               "VALUES (%s)")
            package_data = package_TMP
            cursor.execute(add_package_TMP, package_data)
            cnx.commit()

        except:
            pass

        link_parts = p.split('/')

        Subcategory = link_parts[3]
        Category_TMP = Subcategory

        Titles_TMP = title.strip()

        Prices_TMP = (price.strip().replace('ریال', '').strip())
        Prices_TMP = (Prices_TMP.replace(',',''))
        Prices_TMP = int(Prices_TMP)

        Quaintities_TMP = (quantity.strip())

        Subcategories_TMP = Subcategory

        Link_TMP = p.strip()
        N_of_Links_extracted += 1
        print(f'Links Extracted: {N_of_Links_extracted}')


        add_Product = ("INSERT INTO products "
                       "(Title, Price, Category, Quaintities, Link) "
                       "VALUES (%s, %s, %s, %s, %s)")
        Product_data = (Titles_TMP, Prices_TMP, Category_TMP, Quaintities_TMP, Link_TMP)
        try:
            cursor.execute(add_Product, Product_data)
            cnx.commit()
            print("Bilakh")
        except:
            print("Duplicate")



    except AttributeError:
        numberOfnoQuantity += 1
        print(f'Links Not Extracted: {numberOfnoQuantity}')

cursor.close()
cnx.close()




5468
Links Extracted: 1
Duplicate
Links Extracted: 2
Duplicate
Links Extracted: 3
Duplicate
Links Extracted: 4
Duplicate
Links Extracted: 5
Duplicate
Links Extracted: 6
Duplicate
Links Extracted: 7
Duplicate
Links Extracted: 8
Duplicate
Links Extracted: 9
Duplicate
Links Extracted: 10
Duplicate
Links Extracted: 11
Duplicate
Links Extracted: 12
Duplicate
Links Extracted: 13
Duplicate
Links Extracted: 14
Duplicate
Links Extracted: 15
Duplicate
Links Extracted: 16
Duplicate
Links Extracted: 17
Duplicate
Links Extracted: 18
Duplicate
Links Extracted: 19
Duplicate
Links Extracted: 20
Duplicate
Links Extracted: 21
Duplicate
Links Extracted: 22
Duplicate
Links Extracted: 23
Duplicate
Links Not Extracted: 1
Links Extracted: 24
Bilakh
Links Extracted: 25
Bilakh
Links Extracted: 26
Bilakh
Links Extracted: 27
Bilakh
Links Extracted: 28
Bilakh
Links Extracted: 29
Bilakh
Links Extracted: 30
Bilakh
Links Extracted: 31
Bilakh
Links Extracted: 32
Bilakh
Links Extracted: 33
Bilakh
Links Extracted: 34
B

KeyboardInterrupt: 

In [8]:
  try:
    connection_pool = pooling.MySQLConnectionPool(pool_name="pynative_pool",
                                                  pool_size=3,
                                                  pool_reset_session=True,
                                                  host='localhost',
                                                  database='shop',
                                                  user='root',
                                                  password='00980351zmz')

    print("Printing connection pool properties ")
    print("Connection Pool Name - ", connection_pool.pool_name)
    print("Connection Pool Size - ", connection_pool.pool_size)

    # Get connection object from a pool
    connection_object = connection_pool.get_connection()

    if connection_object.is_connected():

        cursor = connection_object.cursor()
        add_Product = ("INSERT INTO products "
                       "(Title, Packages, Price, Category, Quaintities, Link) "
                       "VALUES (%s, %s, %s, %s, %s, %s)")
        Product_data = (Titles_TMP, Prices_TMP, Category_TMP, Quaintities_TMP, Link_TMP)
        cursor.execute(add_Product, Product_data)

except Error as e:
    print("Error while connecting to MySQL using Connection pool ", e)
finally:
    # closing database connection.
    if connection_object.is_connected():
        cursor.close()
        connection_object.close()
        print("MySQL connection is closed")

['https://ickala.com/interfaces-parts/35154-rm9003b.html', 'https://ickala.com/regulators/43002-78l05-sot89-43002.html', 'https://ickala.com/linear-regulators/9447-ams1117-33.html', 'https://ickala.com/linear-regulators/9448-ams1117-50.html', 'https://ickala.com/regulators/43008-78l05-to92-43008.html', 'https://ickala.com/voltage-current-regulators/11808-78l05-sot23.html', 'https://ickala.com/scr-parts/42406-bt169d-01.html', 'https://ickala.com/holtek-series/42629-holtek-ht66f0025.html', 'https://ickala.com/ic-comarators/24740-lm311dr.html', 'https://ickala.com/ic-comarators/26358-lm393m-copy.html', 'https://ickala.com/buffers-and-line-drivers/18951-74hc125d.html', 'https://ickala.com/display-drivers-parts/2919-ws2811.html', 'https://ickala.com/holtek-series/42627-holtek-bs86d20a-3.html', 'https://ickala.com/regulators/43148-l7805.html', 'https://ickala.com/ic-comarators/2609-lm339dr.html', 'https://ickala.com/attiny-series/35520-attiny13a-35520.html', 'https://ickala.com/linear-regula